In [ ]:
%matplotlib inline

import re
import os
import warnings
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
import numpy as np
import pandas as pd
import seaborn as sns
from pandas.errors import SettingWithCopyWarning
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler


In [ ]:
from sklearn.manifold import TSNE

strategies = ['None', 'Specific', 'Generic']
temperatures = ['0.0', '0.7']
variations = ['0', '1', '2']
recursions = ['0', '1']

regex_patterns = {
    'Powderkg': r'Powderkg = (\d+)',
    'wc': r'wc = ([\d.]+)',
    'materials': r'materials = ([\d./]+)',
    'curing': r'curing = (.+?)(?:\n|$)'
}


def find_explained_variance_using_pca(correlation_matrix, feature, strategy, temperature):
    pca = PCA(n_components=1)
    data = pd.DataFrame(correlation_matrix)
    pca.fit(data)
    # Get the first principal component
    first_principal_component = pca.components_[0]
    # Print the explained variance ratio, which tells you the proportion of variance
    # explained by the first principal component.
    explained_variance_ratio = pca.explained_variance_ratio_
    print(f"Explained Variance Ratio for {feature}, {strategy} and {temperature}:", explained_variance_ratio)
    # Print the first principal component, which represents the overall relationship
    print(f"First Principal Component (Overall Relationship) for {feature}, {strategy} and {temperature}:",
          first_principal_component)


def perform_correlation_analysis(feature, strategy, temperature, variation, recursion):
    correlation_matrix = df.corr()
    correlation_matrix = correlation_matrix.apply(lambda x: round(x, 2))
    average_cc_per_run = correlation_matrix.mean()
    correlation_matrix['Average Correlation'] = average_cc_per_run
    print('AVERAGE CORRELATION COEFFICIENT PER RUN')
    print(average_cc_per_run)
    overall_average_cc = correlation_matrix.stack().mean()
    print('OVERALL CORRELATION COEFFICIENT')
    print(overall_average_cc)
    # NaNs occur when the std is 0, i.e. we have constant functions. We handle this cas by replacing NaN with 1
    correlation_matrix.fillna(1, inplace=True)
    correlation_matrix.to_csv(
        f'Results/Correlation_Analysis/correlation_matrix_{feature}_{strategy}_{variation}_{temperature}_{recursion}.csv')
    plt.figure(figsize=(8, 6))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5)
    # Customize the plot (add labels, title, etc. if needed)
    plt.title(f'Correlation Heatmap {strategy} {temperature}')
    plt.show()
    # plt.savefig(
    #     f'Results/Correlation_Analysis/images/correlation_matrix_{feature}_{strategy}_{variation}_{temperature}_{recursion}.png',
    #     bbox_inches='tight')
    return correlation_matrix


def plot_individual_experiments(feature, strategy, temperature):
    plt.figure(figsize=(8, 6))
    plt.plot(df.index, df[f'{feature} 1'], label=f'{feature} Experiment 1')
    plt.plot(df.index, df[f'{feature} 2'], label=f'{feature} Experiment 2')
    plt.plot(df.index, df[f'{feature} 3'], label=f'{feature} Experiment 3')
    plt.legend()
    plt.title(f'Lineplot for {strategy} and {temperature}')
    plt.xlabel(feature)
    plt.tight_layout()
    plt.show()


def extract_data_for_given_run(pattern):
    matching_filenames = []
    # Iterate through the files in the directory
    dir_name = 'Results/ID'
    for filename in os.listdir(dir_name):
        match = re.match(pattern, filename)
        if match:
            matching_filenames.append(os.path.join(dir_name, filename))
    all_results_df = pd.DataFrame()
    warnings.filterwarnings("ignore", category=SettingWithCopyWarning)
    for i, filename in enumerate(matching_filenames):
        results_sample_df = pd.read_csv(filename)
        for col, pattern in regex_patterns.items():
            all_results_df[f'{col} {i}'] = results_sample_df['Formulation'].str.extract(pattern, expand=False)
            if col.startswith('materials'):
                all_results_df[f'{col} {i}'] = all_results_df[f'{col} {i}'].str[:3].astype(float)
            if col.startswith('curing'):
                all_results_df[f'{col} {i}'][all_results_df[f'{col} {i}'].astype(str).str.startswith('Heat')] = 0
                all_results_df[f'{col} {i}'][all_results_df[f'{col} {i}'].astype(str).str.startswith('heat')] = 0
                all_results_df[f'{col} {i}'][all_results_df[f'{col} {i}'].astype(str).str.startswith('Ambient')] = 1
                all_results_df[f'{col} {i}'][all_results_df[f'{col} {i}'].astype(str).str.startswith('ambient')] = 1
    return all_results_df, matching_filenames


def tsne_for_feature(feature, strategy, temperature):
    X_2d = tsne.fit_transform(df_transpose)
    # Create a scatter plot of the t-SNE results
    plt.figure(figsize=(8, 6))
    plt.scatter(X_2d[:, 0], X_2d[:, 1], c='r', marker='o', label=feature)
    plt.title(f"t-SNE Visualization {strategy} {temperature}")
    plt.legend()
    plt.show()


def tsne_for_all_features(all_features_for_tsne, strategy, temperature):
    scaler = MinMaxScaler()
    scaler.fit(all_features_for_tsne)
    # Transform your DataFrame to apply the scaling
    scaled = pd.DataFrame(scaler.transform(all_features_for_tsne))
    tsne = TSNE(n_components=2, perplexity=2, random_state=0)
    X_2d = tsne.fit_transform(scaled)
    # Create a scatter plot of the t-SNE results
    plt.figure(figsize=(8, 6))
    plt.scatter(X_2d[:, 0], X_2d[:, 1], c='r', marker='o', label='All Features')
    plt.title(f"t-SNE Visualization all features {strategy} {temperature}")
    plt.legend()
    plt.show()


def pca_and_corr_on_all_features(df):
    correlation_matrix = df.corr()
    correlation_matrix = correlation_matrix.apply(lambda x: round(x, 8))
    plt.figure(figsize=(8, 6))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5)
    # Customize the plot (add labels, title, etc. if needed)
    plt.title('Correlation Heatmap between Lineplots')
    plt.show()
    pca = PCA(n_components=1)
    data = pd.DataFrame(correlation_matrix)
    pca.fit(data)
    # Get the first principal component
    first_principal_component = pca.components_[0]
    # Print the explained variance ratio, which tells you the proportion of variance
    # explained by the first principal component.
    explained_variance_ratio = pca.explained_variance_ratio_
    print("Explained Variance Ratio:", explained_variance_ratio)
    # Print the first principal component, which represents the overall relationship
    print("First Principal Component (Overall Relationship):", first_principal_component)

In [ ]:
from itertools import product

all_combinations = list(product(strategies, variations, temperatures, recursions))

#### Correlation Analyses between different runs

In [ ]:
for combination in all_combinations:
    strategy = combination[0]
    variation = combination[1]
    temperature = combination[2]
    recursion = combination[3]
    print('#####################################################################################')
    print(
        f'######### Run with strategy {strategy}, variation {variation}, temperature {temperature} and recursion {recursion} #########')
    print('#####################################################################################')
    pattern = fr'gpt-3.5-turbo_{strategy} {variation}_prompt_experiment_(\d+)_temp_{temperature}_target_(\d+)_\%_Dev_Budget_(\d+)_recursive_{recursion}_(\d+)\.csv'
    all_results_df, matching_filenames = extract_data_for_given_run(pattern)

    warnings.filterwarnings("default")

    features = ['Powderkg', 'wc', 'materials']
    for feature in features:
        print(f'################ Feature {feature} ################')
        data = {}
        for i in range(len(matching_filenames)):
            data[f'{feature} {i}'] = list(all_results_df[f'{feature} {i}'].values)

        df = pd.DataFrame(data)
        correlation_matrix = perform_correlation_analysis(feature, strategy, temperature, variation, recursion)
        # plot_individual_experiments(feature, strategy, temperature)
        find_explained_variance_using_pca(correlation_matrix, feature, strategy, temperature)

        tsne = TSNE(n_components=2, perplexity=2, random_state=0)
        df_transpose = df.T
        if feature == 'Powderkg':
            all_features_for_tsne = df_transpose.values
        else:
            all_features_for_tsne = np.hstack((all_features_for_tsne, df_transpose.values))
        # Comment in in case you want to have a tsne visualisation
        # tsne_for_feature(feature, strategy, temperature)

    # Comment in in case you want to have a tsne visualisation
    # tsne_for_all_features(all_features_for_tsne, strategy, temperature)

    df = pd.DataFrame(all_features_for_tsne).T
    df.head(30)

    scaler = MinMaxScaler()
    scaler.fit(df)

    # Transform your DataFrame to apply the scaling
    scaled_df = pd.DataFrame(scaler.transform(df), columns=df.columns)

    plt.plot(scaled_df.index, scaled_df[1].astype(str))
    plt.plot(scaled_df.index, scaled_df[2].astype(str))
    plt.gca().invert_yaxis()
    plt.show()

    # pca_and_corr_on_all_features(df)

#### Distribution analysis

In [ ]:
from math import floor
from itertools import product

all_combinations = list(product(strategies, temperatures, recursions))


def plot_distribution(df):
    print(df)
    plt.hist(df[0], bins=10, color='skyblue', edgecolor='black')
    plt.xlabel('Values')
    plt.ylabel('Frequency')
    plt.title('Histogram of Values')
    plt.grid(True)
    plt.show()


def compute_std_over_different_runs(df, number_of_iterations=10):
    df = df.copy()
    std_for_each_run = []
    for i in range(number_of_iterations):
        df[i] = df[i].fillna(df[i].astype(float).mean())
        max = df[i].astype(float).max()
        min = df[i].astype(float).min()
        if max - min == 0:
            std = df[i].astype(float).std()
        else:
            std = df[i].astype(float).std() / (max - min)
        std_for_each_run.append(std)
    return np.array(std_for_each_run)


def compute_overall_std(df_1, df_2, df_3, df_4, number_of_iterations=10):
    df_1 = df_1.copy()
    df_2 = df_2.copy()
    df_3 = df_3.copy()
    df_4 = df_4.copy()

    std_for_each_run = []
    recipe_vectors_for_each_run = []

    for i in range(number_of_iterations):
        df_1[i] = df_1[i].fillna(df_1[i].astype(float).mean())
        df_2[i] = df_2[i].fillna(df_2[i].astype(float).mean())
        df_3[i] = df_3[i].fillna(df_3[i].astype(float).mean())
        df_4[i] = df_4[i].fillna(df_4[i].astype(float).mean())

        # TODO: CHECK
        # data = np.array([df_1[i].astype(float).values,
        #                  df_2[i].astype(float).values,
        #                  df_3[i].astype(float).values,
        #                  df_4[i].astype(float).values])
        # sample_variance = np.var(data, axis=0, ddof=1)
        # std_for_each_run.append(sample_variance / (15 * np.max(sample_variance)))

        max_1 = df_1[i].astype(float).max()
        if max_1 == 0:
            max_1 = 1
        df_1[i] = df_1[i].astype(float) / max_1
        square_1 = np.square(df_1[i].values)

        max_2 = df_2[i].astype(float).max()
        if max_2 == 0:
            max_2 = 1
        df_2[i] = df_2[i].astype(float) / max_2
        square_2 = np.square(df_2[i].values)

        max_3 = df_3[i].astype(float).max()
        if max_3 == 0:
            max_3 = 1
        df_3[i] = df_3[i].astype(float) / max_3
        square_3 = np.square(df_3[i].values)
        max_4 = df_4[i].astype(float).max()
        if max_4 == 0:
            max_4 = 1
        df_4[i] = df_4[i].astype(float) / max_4
        square_4 = np.square(df_4[i].values)

        overall_std = np.sqrt(np.std(square_1) + np.std(square_2) + np.std(square_3) + np.std(square_4))
        euclidean_recipe_value = np.sqrt(np.mean(square_1) + np.mean(square_2) + np.mean(square_3) + np.mean(square_4))
        std_for_each_run.append(overall_std)
        recipe_vectors_for_each_run.append(euclidean_recipe_value)
    return recipe_vectors_for_each_run, std_for_each_run


def extract_data_for_given_run_for_all_variations(pattern_0, pattern_1, pattern_2):
    matching_filenames = []
    # Iterate through the files in the directory
    dir_name = 'Results/ID'
    for filename in os.listdir(dir_name):
        match = re.match(pattern_0, filename) or re.match(pattern_1, filename) or re.match(pattern_2, filename)
        if match:
            matching_filenames.append(os.path.join(dir_name, filename))
    all_results_df = pd.DataFrame()
    warnings.filterwarnings("ignore", category=SettingWithCopyWarning)
    for i, filename in enumerate(matching_filenames):
        results_sample_df = pd.read_csv(filename)

        for col, pattern in regex_patterns.items():
            all_results_df[f'{col} {i}'] = results_sample_df['Formulation'].str.extract(pattern, flags=re.DOTALL,
                                                                                        expand=False)
            if col.startswith('materials'):
                all_results_df[f'{col} {i}'] = all_results_df[f'{col} {i}'].str[:3].astype(float)
            if col.startswith('curing'):
                all_results_df[f'{col} {i}'][all_results_df[f'{col} {i}'].astype(str).str.startswith('Heat')] = 0
                all_results_df[f'{col} {i}'][all_results_df[f'{col} {i}'].astype(str).str.startswith('heat')] = 0
                all_results_df[f'{col} {i}'][all_results_df[f'{col} {i}'].astype(str).str.startswith('Ambient')] = 1
                all_results_df[f'{col} {i}'][all_results_df[f'{col} {i}'].astype(str).str.startswith('ambient')] = 1

    return all_results_df, matching_filenames


configs_with_less_than_ten_iterations = 0
std_statistics = pd.DataFrame({
    'configuration': [],
    'mean_std': [],
    'max_std': [],
    'min_std': [],
    'avg_std_gradient': [],
    'initial_std': [],
    'halftime_std': [],
    'final_std': []
})


def _compute_avg_grad(vector):
    diffs = np.array([])
    for i in range(len(vector)):
        if i < len(vector) - 1:
            grad = vector[i + 1] - vector[i]
            diffs = np.append(diffs, grad)
    return np.mean(diffs)

def count_unique_tsne_points(tsne_result):
    precision = 2  # Adjust this based on your data and desired precision
    rounded_coordinates = np.round(tsne_result, precision)

    coordinate_tuples = [tuple(coord) for coord in rounded_coordinates]

    unique = {coord: coordinate_tuples.count(coord) for coord in set(coordinate_tuples)}

    return len(unique.items())

def compute_avg_tsne_dist(tsne_result):
    pairwise_distances = pdist(tsne_result)

    return np.mean(pairwise_distances)

# change to display recipe vector instead
full = True

for combination in all_combinations[:4]:
    strategy = combination[0]
    temperature = combination[1]
    recursion = combination[2]

    print('#####################################################################################')
    print(f'####### Run with strategy {strategy}, temperature {temperature} and recursion {recursion} #########')
    print('#####################################################################################')

    pattern_0 = fr'gpt-3.5-turbo_{strategy} 0_prompt_experiment_(\d+)_temp_{temperature}_target_(\d+)_\%_Dev_Budget_(\d+)_recursive_{recursion}_(\d+)\.csv'
    pattern_1 = fr'gpt-3.5-turbo_{strategy} 1_prompt_experiment_(\d+)_temp_{temperature}_target_(\d+)_\%_Dev_Budget_(\d+)_recursive_{recursion}_(\d+)\.csv'
    pattern_2 = fr'gpt-3.5-turbo_{strategy} 2_prompt_experiment_(\d+)_temp_{temperature}_target_(\d+)_\%_Dev_Budget_(\d+)_recursive_{recursion}_(\d+)\.csv'
    all_results_df, matching_filenames = extract_data_for_given_run_for_all_variations(pattern_0, pattern_1, pattern_2)

    # TODO: properly handle this case
    number_of_iterations = len(all_results_df)
    if not matching_filenames or number_of_iterations != 10:
        print(f'Warning: Some of the files for {strategy} {temperature} {recursion} have less than 10 rows per dataframe')
        configs_with_less_than_ten_iterations = configs_with_less_than_ten_iterations + 1
        if strategy == 'Specific' and temperature == '0.0' and recursion == '0':
            print('\n'.join(matching_filenames))


    warnings.filterwarnings("default")

    features = ['Powderkg', 'wc', 'materials', 'curing']
    powders = {}
    wc_ratios = {}
    materials = {}
    curing = {}

    for feature in features:
        for i in range(len(matching_filenames)):
            if feature == 'Powderkg':
                powders[f'{feature} {i}'] = list(all_results_df[f'{feature} {i}'].values)
            elif feature == 'wc':
                wc_ratios[f'{feature} {i}'] = list(all_results_df[f'{feature} {i}'].values)
            elif feature == 'materials':
                materials[f'{feature} {i}'] = list(all_results_df[f'{feature} {i}'].values)
            elif feature == 'curing':
                curing[f'{feature} {i}'] = list(all_results_df[f'{feature} {i}'].values)

    # TODO: Check plots which show weird behaviour in detail
    df_powders = pd.DataFrame(powders).T
    df_wc = pd.DataFrame(wc_ratios).T
    df_materials = pd.DataFrame(materials).T
    df_curing = pd.DataFrame(curing).T
    # plot_distribution(df_powders)
    # plot_distribution(df_wc)
    # plot_distribution(df_materials)

    ####### TSNE #######

    # Wir fitten den Scaler auf allen Iterationen so dass wir verschiedene Durchläufe in einem Plot miteinander vergleichen können
    # 10 -> number of iterations

    max_powders = np.max(df_powders.astype(float))
    if max_powders == 0:
        max_powders = 1
    max_wc = np.max(df_wc.astype(float))
    if max_wc == 0:
        max_wc = 1
    max_materials = np.max(df_materials.astype(float))
    if max_materials == 0:
        max_materials = 1
    max_curing = np.max(df_curing.astype(float))
    if max_curing == 0:
        max_curing = 1

    df_recipe_vectors = pd.DataFrame()
    plt.figure(figsize=(9, 6))
    for i in range(df_powders.shape[1]):
        df_powders[i] = df_powders[i].fillna(df_powders[i].astype(float).mean())
        df_wc[i] = df_wc[i].fillna(df_wc[i].astype(float).mean())
        df_materials[i] = df_materials[i].fillna(df_materials[i].astype(float).mean())
        df_curing[i] = df_curing[i].fillna(df_curing[i].astype(float).mean())

        # Show optimal recipe only at the -> otherwise we would display it multiple times
        # Original Recipe
        # optimal_recipe = 360,0.5,"0.5/0.5","Heat curing"
        # Encoded original Recipe
        optimal_recipe = [360 / max_powders, 0.5 / max_wc, 0.5 / max_materials, 0]

        df_recipe_vectors['powders'] = df_powders[i].astype(float).values / max_powders - optimal_recipe[0]
        df_recipe_vectors['wc'] = df_wc[i].astype(float).values / max_wc - optimal_recipe[1]
        df_recipe_vectors['materials'] = df_materials[i].astype(float).values / max_materials - optimal_recipe[2]
        df_recipe_vectors['curing'] = df_curing[i].astype(float).values / max_curing - - optimal_recipe[3]

        for j in range(df_recipe_vectors.shape[0]):
            distance_to_optimal = np.linalg.norm(df_recipe_vectors.loc[j,:].values)
            if distance_to_optimal < 0.1:
                print(df_recipe_vectors.loc[j,:].values)
                print(distance_to_optimal)

        # df_recipe_vectors = pd.DataFrame(df_recipe_vectors.T, columns=list(np.arange(15)))
        df_recipe_vectors = df_recipe_vectors.dropna(axis=1)
        # print(df_recipe_vectors.shape[0])
        # print(df_recipe_vectors.shape[1])

        tsne = TSNE(n_components=2, perplexity=8, random_state=42, n_iter=1000)
        tsne_result = tsne.fit_transform(df_recipe_vectors)
        unique_count = count_unique_tsne_points(tsne_result)
        avg_dist = compute_avg_tsne_dist(tsne_result)

        plt.scatter(tsne_result[:, 0], tsne_result[:, 1], label=f'Iteration {i} ({unique_count, avg_dist})')
        plt.scatter(0, 0, marker='x', color='red', label='Optimal Recipe')
        plt.xlabel('TSNE - x')
        plt.ylabel('TSNE - y')
        plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
        plt.title('TSNE along with unique recipes and their avg distance')
        plt.show()
    # plt.savefig(f'Results/Recipe_Distribution_Analysis/TSNE for Strategy {strategy} Temperature {temperature} Recursion {recursion}.png', bbox_inches='tight')

    ####### Standard Deviations #######
    std_for_each_run_powder = compute_std_over_different_runs(df_powders, number_of_iterations)
    std_for_each_run_wc = compute_std_over_different_runs(df_wc, number_of_iterations)
    std_for_each_run_materials = compute_std_over_different_runs(df_materials, number_of_iterations)
    std_for_each_run_curing = compute_std_over_different_runs(df_curing, number_of_iterations)

    # TODO:
    recipe_vector_mean, std_for_recipe_vector = compute_overall_std(df_powders, df_wc, df_materials, df_curing, number_of_iterations)

    full_std = std_for_each_run_powder + std_for_each_run_wc + std_for_each_run_materials + std_for_each_run_curing

    if full:
        std_for_statistics = full_std
    else:
        std_for_statistics = std_for_recipe_vector

    # comment in to see details on the dfs
    # print(df_powders)
    # print(df_materials)
    # print(df_wc)
    # print(df_curing)

    std_statistics.loc[len(std_statistics)] = {
        'configuration': f'{strategy} {temperature} {recursion}',
        'mean_std': np.mean(std_for_statistics),
        'max_std': np.max(std_for_statistics),
        'min_std': np.min(std_for_statistics),
        'avg_std_gradient': _compute_avg_grad(std_for_statistics),
        'initial_std': std_for_statistics[0],
        'halftime_std': std_for_statistics[floor(number_of_iterations / 2)],
        'final_std': std_for_statistics[number_of_iterations - 1],
    }

    nr_iterations = np.arange(0, number_of_iterations)
    plt.figure(figsize=(9, 6))
    plt.plot(nr_iterations, std_for_each_run_powder, label='powderkg')
    plt.plot(nr_iterations, std_for_each_run_wc, label='wc')
    plt.plot(nr_iterations, std_for_each_run_materials, label='materials')
    plt.plot(nr_iterations, std_for_each_run_curing, label='curing')


    if full:
        plt.plot(nr_iterations, full_std, label='Full Std')
    else:
        plt.plot(nr_iterations, recipe_vector_mean, label='Overall Mean')
        plt.plot(nr_iterations, std_for_recipe_vector, label='Overall Std')
        plt.fill_between(np.array(nr_iterations), np.array(recipe_vector_mean) - 0.25 * np.array(std_for_recipe_vector),
                         np.array(recipe_vector_mean) + 0.25 * np.array(std_for_recipe_vector), color='yellow', alpha=0.3,
                         label='Std')

    plt.legend()
    plt.xlabel('Iteration')
    plt.ylabel('Standard Deviation')
    plt.title(f'Std evolution for {strategy} {temperature} {recursion}')

    plt.show()
#     if full:
#         plt.savefig(f'Results/Recipe_Distribution_Analysis/Full Std evolution for Strategy {strategy} Temperature {temperature} Recursion {recursion}.png', bbox_inches='tight')
#     else:
#         plt.savefig(f'Results/Recipe_Distribution_Analysis/Std evolution for Strategy {strategy} Temperature {temperature} Recursion {recursion}.png', bbox_inches='tight')
#
# if full:
#     std_statistics.to_csv('Results/Recipe_Distribution_Analysis/Full std_statistics.csv')
# else:
#     std_statistics.to_csv('Results/Recipe_Distribution_Analysis/std_statistics.csv')

print(f'####### Nr of warnings: {configs_with_less_than_ten_iterations} #######')

# TODO: FIX
# START
#####################################################################################
####### Run with strategy Specific, temperature 0.0 and recursion 0 #########